# Análisis exploratorio inicial

Como se comento en una unidad anterior, en *Python* se dispone de lo que se conoce como *Dataframes* a través de la librería `pandas`. Estos *Dataframes*, como se avanzó previamente, permiten realizar una primera exploración y consultas de una manera muy sencilla. Veamos un ejemplo.

El primer paso siempre es la carga de las librerías que se van a usar:

In [ ]:
# Importamos las librerías que vamos a usar lo primero
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)
import pandas as pd
%matplotlib inline 
#necesario solamente para jupyter notebook
#indicamos a matplotlib el backend que queremos que use

Para este ejemplo vamos a utilizar un toy-dataset de los más conocidos de `scikit-learn` pero nos valdría cualquiera.

Un punto a destacar es que al tratarse de un dataset tan conocido, scikit-learn lo incluye por defecto en los datasets que incorpora, por lo que lo cargaremos haciendo uso de las librerías. En un entorno de análisis real, cargaríamos los datos desde un fichero csv, excel, SPSS, matlab, imágenes...

Por defecto los datasets que vienen cargados en `scikit-learn` no están hechos con `pandas` y son conocidos como bunch. Los *Dataframes* vienen con muchas opciones por defecto que hacen que merezca la pena tomarse la molestia de convertirlos para aprovechar al máximo la potencia de la librería. Como además esto es algo que puede que tengamos que hacer muchas veces, vamos a definir una función en python que me convierta de manera genérica cualquier dataset de scikit-learn en un *Dataframe* de `pandas`.

In [ ]:
from sklearn import datasets

def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

df_iris = sklearn_to_df(datasets.load_iris())

df_iris.head()

Ya lo hemos visto en unidades anteriores, pero en este tipo de problemas contamos con la descripción. S bien tenemos otras opciones que pueden ser útiles como:

* df_iris.shape
* df_iris.index
* df_iris.columns
* df_iris.info()
* df_iris.count()
* df_iris.sum()
* df_iris.cumsum()
* df_iris.min() 
* df_iris.max()
* df_iris.idxmin()
* df_iris.idxmax()
* df_iris.describe()
* df_iris.mean()
...

Veamos un ejemplo:

In [ ]:
tmp = datasets.load_iris()

#Esta librería es solo para mejorar la lectura de los resultados
from rich import print

print(tmp.DESCR)

In [ ]:
 df_iris.idxmax()

Otro de los elemenos que nos permiten es aplicar funciones sobre el dataset o hacer consultas sobre el mismo como:

In [ ]:
function = lambda x: x*2
df_iris.apply(function)

In [ ]:
df_iris.target.unique()

Una de las mejores cosas de *Python* es que a los paquetes, se les suele añadir una cheat sheet, que es un resumen gráfico de las opciones más comunes que se suelen utilizar. Por ejemplo, para pandas la tenemos disponible [aquí](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf).  

Por lo tanto, antes de plantearnos siquiera realizar cualquier análisis, un data scientist debe preguntarse lo siguiente, ¿conozco mis datos? a pesar de que los análisis comparten fases y se pueden elaborar pipelines de análisis que se mantienen en el tiempo, no sirve ABSOLUTAMENTE de nada analizar unos datos y no saber interpretar los resultados. También es conveniente conocer de manera, lo más exhaustiva posible.

Vemos algo de esa información en los siguientes apartado:

In [ ]:
df_iris.shape

In [ ]:
df_iris.info()

In [ ]:
df_iris.describe()

In [ ]:
df_iris.target.value_counts()

In [ ]:
df_iris["target"].value_counts()

In [ ]:
df_iris.boxplot(by="target",figsize=(10,10))

In [ ]:
df_iris.plot(kind='box')

In [ ]:
df_iris.hist(edgecolor='black')

In [ ]:
df_iris.hist(bins=100,figsize=(10,10))
#plt.show() # en jupyter es opcional, no hace falta. Muestra plots por defecto

In [ ]:
df_iris_toplot=df_iris.drop('target',axis=1)
df_iris_toplot.hist(edgecolor='red')

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(df_iris_toplot,figsize=(10,10))
plt.show()

In [ ]:
sns.pairplot(data=df_iris_toplot)

In [ ]:
sns.pairplot(df_iris, hue="target")

In [ ]:
#problema
#sns.pairplot(dataset, hue="target")
#ver ayuda de la función

df_iris.loc[df_iris.target==0,'target_cat']='setosa'
df_iris.loc[df_iris.target==1,'target_cat']='versicolor'
df_iris.loc[df_iris.target==2,'target_cat']='virginica'
print(df_iris)

nombre_columnas=list(df_iris.columns)

In [ ]:
sns.pairplot(df_iris.iloc[:,[0,1,2,3,5]],hue='target_cat',diag_kind="hist")

# Mejorando la exploración

Por desgracia, no hay nada en scikit-learn para gestionar *DataFrames* de `pandas` directamente y es un poco tedioso trabajar únicamente con *numpy* arrays, la mayoría de las veces se tarda mucho tiempo en conseguir obtener la misma información de un *numpy* por tener que programar explícitamente los gráficos. Es por esto que se suele realizar la transformación a *dataframes* haciendo el análisis gráfico y descriptivo, pero si usamos las funciones de scikit-learn, estamos saltando continuamente. Además, en general, nos interesa hacer los plots descriptivos solamente de los atributos numéricos.

Es por ello que intentaremos realizar algunas de las operaciones ya sobre los propios datos como veremos, así como algunos de los plots, pero sed muy concientes de que esto se hace sobre los `numpy.array` no sobre  los `pandas.dataframe`

## Búsqueda de correlaciones

Para esta parte del tutorial nos basaremos en un dataset ya preparado, *California housing prices*. Este es un modelo no tan conocido como el *Boston housing prices* pero al contrario que este último no plantea cuestiones de sesgo racial. En primer lugar procederemos a cargar la base de datos en cuestión.

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing(as_frame=True)

# Descipción del modelo a tratar con el nuemero de variables y su significado
print(housing.DESCR)

El siguiente paso es calcular sobre el dataframe la matriz de correlación, es decir, como de vinculadas están las variables entre sí. Para ellos vamos a mostrar los valores con respecto a la media de ingresos en un vecindario.

In [ ]:
matriz_correlacion = housing.frame.corr()
matriz_correlacion["MedInc"].sort_values(ascending=False)

#### Atención!!!
Correlación no implica causalidad. Mucho cuidado con esto en los modelos de regresión. Os recomiendo esta [lectura](https://www.jotdown.es/2016/06/correlacion-no-implica-causalidad/) (spoiler: es lectura divulgativa, por lo que no entrará en la prueba final)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
housing.frame.hist(bins=50,figsize=(15,15))
plt.show()

In [ ]:
from pandas.plotting import scatter_matrix

features = ["MedInc","HouseAge","Population","MedHouseVal","AveRooms"]

scatter_matrix(housing.frame[features],figsize=(15,15))

En lugar de pintar en la diagonal la correlación de una variable consigo misma, lo que pinta es el histograma de cada variable

# Pre-procesado de los datos

Después de hacer un análisis exploratorio inicial como hemos visto en este notebook, llega el momento de prepar los datos para entrenar los modelos de ML. Vamos a ver dos de las acciones más habituales durante el pre-procesado: eliminación de **NA** y escalado de las features y en una unidad posterior abordaremos la selección y reducción de la dimensionalidad:

## Eliminando NANs

In [ ]:
import numpy as np
import pandas as pd

datos = pd.DataFrame([[0, 0, np.nan], [np.nan, 1, 1], [2,2,2]],columns=["A","B","C"])
display(datos) #es el print para jupyter

la mayoría de los modelos de ML no son capaces de trabajar con ausencia de datos, los famosos **NA**. En el otro caso, trabajan con ellos porque preprocesan los datos por nosotros eliminando los **NA** (veremos que hay varias aproximaciones) o haciendo imputación del valor (varias aproximaciones). La recomendación aquí es estudiar los datos previamente y realizar de manera explícita todas las transformaciones necesarias. Primero, porque el control del pre-procesado es explícito y corre a cargo del investigador. Segundo, porque todos los modelos hacen lo mismo (nada asegura que los modelos que pre-procesen los **NA** lo hagan de la misma manera y la comparación no sería justa. Recalcar que en este caso vamos a pasar los datos que tengamos a Dataframe después del examen inicial ya que nos permitirá un tratamiento más semcillo de estás incidencias.

In [ ]:
datos.isna().sum()

Nuestros datos pueden tener NA. ¿Qué opciones tenemos?

1. Quitar los casos en los que tenemos los NA.

   Veasen algunos ejemplos de como lidiar con esta cuestión en los siguientes puntos

In [ ]:
len(datos)

In [ ]:
datos.dropna(subset=["A"])

In [ ]:
datos.dropna(how="any")

In [ ]:
datos.dropna(how="all")

In [ ]:
datos.drop("A",axis=1)

In [ ]:
datos.drop(0,axis="index")

dependiendo del dataset, del tamaño, del números de **NA** por línea o columnas, deberemos tomar decisiones, en ocasiones no nos podemos dar el lujo de perder ejemplos/features, por lo que una buena opción es *imputar* el valor en el hueco. Vamos a ver las opciones en la [web](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

imputer.fit(datos)

datos_imp = imputer.transform(datos)
#devuelve un Numpy array básico

display(pd.DataFrame(datos_imp,columns=datos.columns))

Este no es más que uno de los ejemplos. Otras opciones son el `KNNImputer` que con los datos restantes a donde están los **NA** asignará un valor por proximidad con los ejemplos más similares.

Un punto importante a remarcar, al igual que en el procedimiento que veremos a continuación, es que este se debe de aplicar o ajustar sobre los datos de entrenamiento para posteriormente aplicarlo SIN AJUSTAR a los datos de test. Esto se consigue como se verá un poco más adelante con una llamada al método `tranform` con el que cuentan este tipo de operaciones

## Escalado de features

Al igual que sucedía con la eliminación de los **NA**, como tónica general los modelos de ML no son capaces de generalizar bien si las features del dataset están en dimensiones muy diferentes. Imaginemos un caso en el que una medida está en millones de unidades y otra tiene variaciones en el 4 decimal. Es fácil imaginar con cuál se va a centrar el dataset y no tiene por qué ser la mejor variable (la más descriptiva).

Existen dos aproximaciones que se suelen utilizar principalmente para el escalado de las features: cambiar la escala entre valores mínimo y máximo o estandarización. Por desgracia la traducción de las dos en castellano es **normalización**, pero en inglés diferencian entre *normalization* y *standarization*.

+ Normalización o escalado entre mínimo y máximo: la aproximación habitual hace que los valores pasen a encontrarse en el rango entre 0 y 1. Esto se hace restando el mínimo del rango al valor y diviendo por el máximo menos el mínimo. En scikit-learn existe un método (*transformer*) llamado *MinMaxScaler* para esto que además permite cambiar con un hiperparámetro llamado *feature_range* el valor del rango (por defecto está entre 0 y 1.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

data = [[-1, 2], [-0.5, 6], [0, 10], [1, 18]]

print(f'Datos originales: {data}')

Una vez cargados los datos que sean, el siguiente paso es definir y entrenar (ajustar) el objeto que realizará el escalado en este caso `scaler`

In [ ]:
scaler = MinMaxScaler()
scaler.fit(data)

Como en cualquier otro elemento de `scikit-learn` se pueden consultar alguno de los datos internos encontrados, por ejemplo, en este caso cuales han sido los valores mínimo y máximo de cada columna con los que se procederá a realizar la normalización

In [ ]:
print(f'Datos usados como máximo por columna: {scaler.data_max_}')
print(f'Datos usados como mínimo por columna: {scaler.data_min_}')

Desde este punto ya podemos ver los datos tras la transformación

In [ ]:
print(f'Datos normalizados: {scaler.transform(data)}')

A partir de este punto, se puede ver como afectaría meter un nuevo valor en el rango (incluso alguno que estuviese fuera del inicial).

In [ ]:
print(scaler.transform([[2, 2]]))

PREGUNTA: ¿por qué es importante esto?



todo modelo entrenado con datos escalados, aprende en esa *transformación*, es básico que los nuevos casos que queramos predecir (generalmente aquellos que no conoce el modelo y que usamos para validar la capacidad de generalización del mismo o incluso datos que se pasan al modelo una vez puesto en predicción). Es decir, esto es **crítico**.

PREGUNTA: ¿Qué contesta un modelo cuando le pasamos datos que están fuera el espacio donde ha sido entrenado?




Veamos esto con el ejemplo de housing. 

In [ ]:
housing.frame.plot(kind='box')

Lo primero que llama la atención en cuanto analizamos los datos, es que la diferencia en escala que tenemos entre las medidas. Si no queremos que esto provoque un sesgo a la hora de medir la importancia de las variables en un modelo de machine learning, lo primero que deberemos hacer es normalizar los datos. En este caso usaremos la forma más básica que ya se ha introducido de normalización entre máximo y mínimo.

In [ ]:
scaler = MinMaxScaler()
housing_scaled=scaler.fit_transform(housing.frame)

plt.figure()
plt.boxplot(housing_scaled)

plt.show()

Com se puede ver una vez que se escalan las variables estar capturan la variabilidad dde manera que se pueden ver diferencias entre patrones. Un punto a tener en cuenta es que esta noemalización coloca todos los valores en el intervalo $[0,1]$. En otros casos será preciso por ejemplo aplicar otro tipo de normalización como la puntuación estandar o estandarización que restringe los datos al intervalo $[-1,1]$. Con el mismo ejemplo, este segundo procesado se puede lograr con el siguiente código.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
housing_standard=scaler.fit_transform(housing.frame)

plt.figure()
plt.boxplot(housing_standard)

plt.show()

Con la estandarización nos aseguramos que  los datos están centrados en el cero (lo que se hace es restarles el valor medio) y se dividen entre la varianza, con lo que varianza del conjunto es 1. 

**PREGUNTA: ¿cuál es la principal diferencia con la otra aproximación?**



Evidentemente, los valores no están comprendidos entre ningún rango fijo como antes. Hay modelos de ML que son muy sensibles a una, otra o ambas transformaciones. Por ejemplo las redes de neuronas artificales están esperando valores en las features que estén en el rango entre 0 y 1.

### Ventajas entre las aproximaciones

Partiendo de conocer si al modelo le afecta, deberemos ajustar el escalado al mismo, existe una ventaja principal del uso de una de estas técnicas:

+ Básicamente en el caso de que el dataset tenga outliers, la recomendación es aplicar una estandarización a los datos ya que es mucho más robusta.

### Precauciones

Es importante ajustar los escalados a los datos de entrenamiento únicamente, no a todo el dataset (lo que incluiría el conjunto de test). Lo que hay que hacer es calcular los escalados sobre los datos de entrenamiento y después, ajustar de acuerdo a dicha transformación los datos siguientes (test o nuevos datos). Esto es posible hacerlo con una llamada a la función `transform` que tienen todos los escalados y transformaciones en general.

# Pipelines en scikit-learn

Como ya se ha comentado en una unidad anterior, en machine learning los pasos para crear y entrenar y testear un modelo, habitualmente están claros. A una secuencia de estos pasos se le suele llamar un *pipeline* de procesado y al ser tan comunes, el propio `scikit-learn` lo suele aplicar y nos da una clase de utilidad para el soporte de este tipo de operaciones. 
Los pipelines de scikit-learn son un elemento extremadamente potente de cara a automatizar difernetes acciones que necesitan ser realizadas de manera secuencial. Como se comentó anteriormente puede ser necesario trabajar con *data.frames* por la enorme cantidad de funcionalidades distintas que aporta esta librería. Sin embargo, por cuestiones de velocidad de procesado y de optimización, scikit-learn utiliza numpy arrays.

en todo caso, durante la fase de pre-procesado, una vez se ha tomado la decisión de qué hacer con los **NA** y qué tipo de escalado se va a aplicar, se puede preparar un pipeline. Para ello, `scikit-learn` dispone de la clase *pipeline* lo cual, muy afortunadamente, tiene múltiples usos. A lo largo del curso la usaremos en más ocasiones para automatizar diferentes ejecuciones. Veamos a continuación un ejemplo muy sencillo de uso. En este tomaremos un dataset de juguete con **NANs** de que hemos decidido que se pre-procesará con un pipeline que haga imputación a la media y un escalado Entre el máximo y el mínimo.

Creamos ese dataset de juguete inicial en este caso es más que sencillo con solo 3 patrones en su haber.

In [ ]:
import pandas as pd
import numpy as np
datos_trainn = pd.DataFrame([[0, 0, np.nan], [np.nan, 1, 1], [2,2,2]],columns=["A","B","C"])

datos_test = pd.DataFrame([[3, 0, np.nan], [1, 1, 1], [3,3,3]],columns=["A","B","C"])

display(datos)

Veamos a continuación lo que sería el modelo paso a paso y comparemoslo con el uso de los pipelienes.

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

imputer.fit(datos_train)
datos_train_imp = imputer.transform(datos_train)
datos_test_imp = imputer.transform(datos_test)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
datos_train_imp_standard=scaler.fit_transform(datos_train_imp)
datos_test_imp_standard=scaler.transform(datos_test_imp)

display(pd.DataFrame(datos_train_imp_standard, columns=["A","B","C"]))

In [ ]:
display(pd.DataFrame(datos_test_imp_standard, columns=["A","B","C"]))

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

pipeline = Pipeline([
    ('imputacion_nans',SimpleImputer(strategy="median")),
    ('standarizacion',StandardScaler()),
])

datos_train_pipeline = pipeline.fit_transform(datos_train)
datos_test_pipeline = pipeline.transform(datos_test)
display(pd.DataFrame(datos_train_pipeline, columns=["A","B","C"]))

In [ ]:
display(pd.DataFrame(datos_test_pipeline, columns=["A","B","C"]))

como se puede ver, el resultado es el mismo!!

Así mismo también debe de destacarse que, en un pipeline también se podrían incluir transformaciones de columnas o el entrenamiento de los modelos como veremos un poco más adelante en otra unidad.